In [1]:
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv

In [6]:
def pull_from_csv(csv_file, row=3, header=1):
        lwp_ind = str(csv_file).lower().find('lwp')
        lwp = str(csv_file)[lwp_ind:lwp_ind+6]
        df = pd.read_csv(csv_file, header=[header], error_bad_lines=False)
        try:
            return [lwp,*df.iloc[row].name]
        except:
            return [lwp,*df.iloc[row]]
        
        
        
        
        
def met_ratio(df, up, down, error_method='Cramer', error_label=' Error (SD)', output='both'):
    if not isinstance(up,list):
        up = [up]
    if not isinstance(down,list):
        down = [down]    
   
    up_err_name = [f'{item}{error_label}' for item in up]
    down_err_name = [f'{item}{error_label}' for item in down]
    for item in np.concatenate([up_err_name, down_err_name]).flat:
        if item not in df.columns:
            raise KeyError(f'Cannot find "{item}" in the DataFrame. Please check the given name is correct.')
    
    up_amp = df[up].astype(float).sum(axis=1)
    down_amp = df[down].astype(float).sum(axis=1)
    if error_method.lower() == 'cramer':
        up_err = (df[up_err_name].astype(float)**2).sum(axis=1) ** 0.5
        down_err = (df[down_err_name].astype(float)**2).sum(axis=1) ** 0.5
    elif error_method.lower()=='narrow':
        up_err = df[up_err_name].astype(float).max(axis=1) 
        down_err = df[down_err_name].astype(float).max(axis=1) 
    else:
        raise KeyError('Please choose "Cramer" or "Narrow" for the error_method')
    
    ratio = up_amp / down_amp
    ratio_err = ratio * ((up_err/up_amp)**2 + (down_err/down_amp)**2)**0.5 
    
    if len(up)>1:
        up_label =f'({"+".join(up)})'
    else:
        up_label = f'{"+".join(up)}'
    if len(down)>1:
        down_label = f'({"+".join(down)})'
    else:
        down_label = f'{"+".join(down)}'    
    col_name = f'{up_label}/{down_label}'   
    col_error_name = f'{col_name} Error (SD) {error_method}'
    
    if output=='both':
        return pd.DataFrame({col_name: ratio, col_error_name: ratio_err})
    elif output=='ratio':
        return pd.DataFrame({col_name: ratio})      
    elif output=='error':
        return pd.DataFrame({col_error_name: ratio_err})   
    else:
        raise KeyError('Please choose "ratio", "error" or "both" (default) as output')

In [13]:
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY')
data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/Ellie')
data_folder = Path('/Users/Patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA')

basis_folder = Path('/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine')

sdat_files = [f for f in sorted(data_folder.rglob('*')) if ("act.sdat" in f.name.lower() and "PRESS_35_WM".lower() in f.name.lower())]
for file in tqdm(sdat_files):
    ref = Path(f'{str(file)[0:-8]}ref.SDAT')
    csv = Path(f'{str(file)[0:-5]}_PRESS_35_WM.csv')
    pdf = Path(f'{str(file)[0:-4]}pdf')
    # print(csv)
    if not csv.is_file():
        # command = f'tarquin --rows 3 --cols 2 --input {file} --output_pdf {pdf} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        # command = f'tarquin --rows 3 --cols 2 --input {file} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        command = f'tarquin --input {file} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'

        print(f'{command}\n') 
        os.system(command)


  0%|          | 0/57 [00:00<?, ?it/s]

tarquin --input /Users/Patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA/LWP711_morning/C3595760_WIP_SV_PRESS_35_WM_14_1_raw_act.SDAT --output_csv /Users/Patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA/LWP711_morning/C3595760_WIP_SV_PRESS_35_WM_14_1_raw_act_PRESS_35_WM.csv --input_w /Users/Patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA/LWP711_morning/C3595760_WIP_SV_PRESS_35_WM_14_1_raw_ref.SDAT --basis_csv /Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine


2022-Sep-15 12:57:18 INFO   : TARQUIN 4.3.10 Started
2022-Sep-15 12:57:18 INFO   : Loading data file : /Users/Patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA/LWP711_morning/C3595760_WIP_SV_PRESS_35_WM_14_1_raw_act.SDAT
2022-Sep-15 12:57:18 INFO   : Loading data file : /Users/Patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA/LWP711_morning/C3595760_WIP_SV_PRESS_35_WM_14_1_raw_act.SDAT
2022-Sep-15 12:57:18 INFO   : nStart = 20
2022-Sep-15 12:57:18 INFO   : nEnd   = 512
TARQUIN Version 4.3.10

**** TARQUIN is a research tool and is NO

.............


2022-Sep-15 12:57:19 INFO   : done.
2022-Sep-15 12:57:19 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Glc.csv'
2022-Sep-15 12:57:19 INFO   : done.
2022-Sep-15 12:57:19 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Gln.csv'
2022-Sep-15 12:57:19 INFO   : done.
2022-Sep-15 12:57:19 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Glu.csv'

.........................................

...


2022-Sep-15 12:59:09 INFO   : done.
2022-Sep-15 12:59:09 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Glc.csv'
2022-Sep-15 12:59:09 INFO   : done.
2022-Sep-15 12:59:09 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Gln.csv'

.......................................


2022-Sep-15 12:59:09 INFO   : done.
2022-Sep-15 12:59:09 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Glu.csv'
2022-Sep-15 12:59:09 INFO   : done.
2022-Sep-15 12:59:09 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Gua.csv'
2022-Sep-15 12:59:09 INFO   : done.
2022-Sep-15 12:59:09 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Ins.csv'
2022-Sep-15 12:59:09 INFO   : done.
2022-Sep-15 12:59:09 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Lac.csv'
2022-Sep-15 12:59:09 INFO   : done.
2022-Sep-15 12:59:09 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Lip09.csv'
2022-Sep-15 12:59:09 INFO   : done.
2022-Sep-15 12:59:09 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basis_threonine/Lip13a.csv'
2022-Sep-15 12:59:09 INFO   : done.
2022-Sep-15 12:59:09 INFO   : Simulating '/Users/Patxi/Sync/Projects/PAINT_MRS/3_0T_basi

100%|██████████| 57/57 [01:51<00:00,  1.96s/it].........................................................................................................................................


2022-Sep-15 12:59:10 INFO   : done.
2022-Sep-15 12:59:10 DEBUG  : Final Beta: 32.54
2022-Sep-15 12:59:10 DEBUG  : Final Phi0: -0.38
2022-Sep-15 12:59:10 DEBUG  : Final Phi1: 0.00
2022-Sep-15 12:59:10 INFO   : FWHM right ppm = 2.008053
2022-Sep-15 12:59:10 INFO   : FWHM left ppm = 2.069183
2022-Sep-15 12:59:10 INFO   : Metabolite FWHM (ppm) = 0.061130
2022-Sep-15 12:59:10 INFO   : Spec noise            = 0.000001
2022-Sep-15 12:59:10 INFO   : sdev noise = 0.000042
2022-Sep-15 12:59:10 INFO   : SNR residual = 4.842519
2022-Sep-15 12:59:10 INFO   : SNR max      = 6.001302
2022-Sep-15 12:59:10 INFO   : Ymax       = 0.000410
2022-Sep-15 12:59:10 INFO   : Ymax metab = 0.000410
2022-Sep-15 12:59:10 INFO   : Fit quality = 1.239293
2022-Sep-15 12:59:10 INFO   : Baseline dev = 0.050913
2022-Sep-15 12:59:10 INFO   : Ymax         = 0.000410
2022-Sep-15 12:59:10 INFO   : BSL         = 0.104381
2022-Sep-15 12:59:10 INFO   : Computing CRLBs.
2022-Sep-15 12:59:10 DEBUG  : Sigma: 1.163698e-12
2022-Sep

In [14]:
csv_files = [f for f in sorted(data_folder.rglob('*'), key=lambda x:x.parent.parent.stem[4:6]) if ("act_both.csv" in f.name.lower() and "csi" in f.name.lower())]



rows_to_process = {'BGT': 5, #(BGT = Basal Ganglia and Thalamic, ie deep GM)
                   'WM': 3,
                   }

for label,row_num in rows_to_process.items():
    data_list = []
    for file in csv_files:
        data_list.append([*pull_from_csv(file,row=row_num),*pull_from_csv(file,row=row_num+8),*pull_from_csv(file,header=16,row=row_num+1)])
    my_columns = list(pd.read_csv(file, header=[1], error_bad_lines=False))
    my_errors = [item+' Error (SD)' for item in my_columns]
    my_diagnostics = pull_from_csv(file,row=0, header=16)    
    df = pd.DataFrame(data_list, columns=['LWP', *my_columns, 'Error for LWP', *my_errors,*my_diagnostics ])
    df.to_excel(f'{label}.xlsx', index=False)
    
    #now calculate ratios
    df_L2N = met_ratio(df,['Lac', 'Threonine'], 'TNAA')
    df_L2N_narrow_error = met_ratio(df,['Lac', 'Threonine'], 'TNAA', error_method='Narrow', output='error')
    df_N2C = met_ratio(df,'TNAA', 'TCho')
    df_C2C = met_ratio(df,'TCho', 'Cr')
    df_N2Cr = met_ratio(df,'TNAA', 'Cr')


    # df_ratio = pd.concat((df_L2N, df_N2C, df_C2C, df_N2C, df), axis=1)
    df_ratio = pd.concat((df_L2N,df_L2N_narrow_error, df_N2C, df_C2C, df_N2Cr, df), axis=1)

    df_ratio.set_index('LWP').to_excel(f'{label}_RATIOS_PRESS_INSPIRE.xlsx')



/var/folders/qr/hf3x765n1mjgzkb2_by7zrcc0000gx/T/ipykernel_8312/710225691.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(csv_file, header=[header], error_bad_lines=False)
b'Skipping line 18: expected 25 fields, saw 34\nSkipping line 19: expected 25 fields, saw 34\nSkipping line 20: expected 25 fields, saw 34\nSkipping line 21: expected 25 fields, saw 34\nSkipping line 22: expected 25 fields, saw 34\nSkipping line 23: expected 25 fields, saw 34\nSkipping line 24: expected 25 fields, saw 34\n'
/var/folders/qr/hf3x765n1mjgzkb2_by7zrcc0000gx/T/ipykernel_8312/710225691.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(csv_file, header=[header], error_bad_lines=False)
b'Skipping line 18: expected 25 fields, saw 34\nSkipping line 19: expected 25 fields, saw 34\nSkip

In [15]:
dfbb = met_ratio(df,'TNAA', 'TCho', error_method='Narrow', output='error')
dfbb


,TNAA/TCho Error (SD) Narrow
0,1.156009
1,3.540930
2,1.541871
3,1.179148
4,1.038445
5,0.701964
6,1.713380
7,2.189811
8,0.541786
9,1.202693
